# Stock Data Incremental Updater for Kaggle

**Features:**
- Loads Google Drive credentials from Kaggle dataset
- Downloads existing CSV files from Google Drive
- Appends only new trading days since last update
- Uploads updated files back to Google Drive

**Setup Requirements:**
1. Add `google-drive-token` dataset to this notebook (contains token.pickle)
2. Run once - subsequent runs will be incremental

**Google Drive Folder Structure:**
```
StockData/
├── SP500/
│   ├── sp500_adj_close.csv
│   ├── sp500_close.csv
│   ├── sp500_open.csv
│   ├── sp500_high.csv
│   ├── sp500_low.csv
│   └── sp500_volume.csv
└── NASDAQ/
    ├── nasdaq_adj_close.csv
    └── ...
```

In [ ]:
# Install required packages
!pip install yfinance pandas numpy google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client tqdm -q

print("✅ All packages installed")

In [ ]:
# Import libraries
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from tqdm import tqdm
import time
import pickle
import io
import os
from pathlib import Path
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload, MediaIoBaseDownload, MediaIoBaseUpload
from google.auth.transport.requests import Request

print("✅ Libraries imported")

## Configuration

In [ ]:
# ============================================================================
# CONFIGURATION
# ============================================================================

# Google Drive folder name where your stock data is stored
# The script will create this folder if it doesn't exist
DRIVE_FOLDER_NAME = 'StockData'

# Path to token.pickle from Kaggle dataset
# Add your google-drive-token dataset to this notebook
TOKEN_PATH = '/kaggle/input/google-drive-token/token.pickle'

# Date configuration
START_DATE = '2020-01-01'  # Used only if no existing data found
END_DATE = datetime.today().strftime('%Y-%m-%d')

# File types to update
PRICE_TYPES = ['adj_close', 'close', 'open', 'high', 'low', 'volume']

print(f"📁 Google Drive folder: {DRIVE_FOLDER_NAME}")
print(f"📅 End date: {END_DATE}")
print(f"🔑 Token path: {TOKEN_PATH}")

## Load Google Drive Credentials

In [ ]:
# ============================================================================
# LOAD CREDENTIALS FROM KAGGLE DATASET
# ============================================================================

def load_credentials():
    """Load Google Drive credentials from Kaggle dataset"""
    
    if not os.path.exists(TOKEN_PATH):
        print("❌ ERROR: token.pickle not found!")
        print("\n📋 Setup Instructions:")
        print("1. Make sure you've added the 'google-drive-token' dataset to this notebook")
        print("2. Click 'Add Data' → Search for your dataset → Add")
        print("3. Re-run this cell")
        raise FileNotFoundError(f"Token file not found at {TOKEN_PATH}")
    
    print(f"🔑 Loading credentials from {TOKEN_PATH}...")
    
    with open(TOKEN_PATH, 'rb') as token:
        creds = pickle.load(token)
    
    # Refresh token if expired
    if creds and creds.expired and creds.refresh_token:
        print("🔄 Refreshing expired token...")
        creds.refresh(Request())
        print("✅ Token refreshed")
    
    print("✅ Credentials loaded successfully")
    return creds

def get_drive_service():
    """Initialize Google Drive API service"""
    creds = load_credentials()
    service = build('drive', 'v3', credentials=creds)
    print("✅ Google Drive service initialized")
    return service

# Initialize service
drive_service = get_drive_service()

## Google Drive Helper Functions

In [ ]:
# ============================================================================
# GOOGLE DRIVE HELPER FUNCTIONS
# ============================================================================

def find_or_create_folder(service, folder_name, parent_id=None):
    """Find existing folder or create new one in Google Drive"""
    
    # Search for folder
    query = f"name='{folder_name}' and mimeType='application/vnd.google-apps.folder' and trashed=false"
    if parent_id:
        query += f" and '{parent_id}' in parents"
    
    results = service.files().list(
        q=query,
        spaces='drive',
        fields='files(id, name)'
    ).execute()
    
    folders = results.get('files', [])
    
    if folders:
        print(f"✅ Found existing folder: {folder_name} (ID: {folders[0]['id']})")
        return folders[0]['id']
    
    # Create folder if not found
    file_metadata = {
        'name': folder_name,
        'mimeType': 'application/vnd.google-apps.folder'
    }
    
    if parent_id:
        file_metadata['parents'] = [parent_id]
    
    folder = service.files().create(
        body=file_metadata,
        fields='id'
    ).execute()
    
    print(f"📁 Created folder: {folder_name} (ID: {folder['id']})")
    return folder['id']

def find_file_in_folder(service, filename, folder_id):
    """Find a file by name in specific folder"""
    
    query = f"name='{filename}' and '{folder_id}' in parents and trashed=false"
    
    results = service.files().list(
        q=query,
        spaces='drive',
        fields='files(id, name, modifiedTime)'
    ).execute()
    
    files = results.get('files', [])
    return files[0] if files else None

def download_csv_from_drive(service, file_id, filename):
    """Download CSV file from Google Drive and return as DataFrame"""
    
    try:
        request = service.files().get_media(fileId=file_id)
        fh = io.BytesIO()
        downloader = MediaIoBaseDownload(fh, request)
        
        done = False
        while not done:
            status, done = downloader.next_chunk()
        
        fh.seek(0)
        df = pd.read_csv(fh, index_col=0, parse_dates=True)
        print(f"✅ Downloaded {filename}: {df.shape}")
        return df
    
    except Exception as e:
        print(f"❌ Error downloading {filename}: {e}")
        return None

def upload_csv_to_drive(service, df, filename, folder_id, file_id=None):
    """Upload DataFrame as CSV to Google Drive (update if exists, create if new)"""
    
    try:
        # Convert DataFrame to CSV bytes
        csv_buffer = io.BytesIO()
        df.to_csv(csv_buffer)
        csv_buffer.seek(0)
        
        media = MediaIoBaseUpload(
            csv_buffer,
            mimetype='text/csv',
            resumable=True
        )
        
        if file_id:
            # Update existing file
            updated_file = service.files().update(
                fileId=file_id,
                media_body=media
            ).execute()
            print(f"✅ Updated {filename} in Drive")
        else:
            # Create new file
            file_metadata = {
                'name': filename,
                'parents': [folder_id]
            }
            new_file = service.files().create(
                body=file_metadata,
                media_body=media,
                fields='id'
            ).execute()
            print(f"✅ Created {filename} in Drive (ID: {new_file['id']})")
        
        return True
    
    except Exception as e:
        print(f"❌ Error uploading {filename}: {e}")
        return False

print("✅ Google Drive helper functions defined")

## Ticker Lists

In [ ]:
# ============================================================================
# TICKER LISTS (S&P 500 & NASDAQ-100)
# ============================================================================

def get_sp500_tickers():
    """S&P 500 ticker list"""
    tickers = [
        'AAPL', 'MSFT', 'NVDA', 'AMZN', 'GOOGL', 'META', 'GOOG', 'BRK-B', 'AVGO', 'LLY',
        'JPM', 'TSLA', 'V', 'UNH', 'XOM', 'WMT', 'MA', 'COST', 'HD', 'PG',
        'JNJ', 'NFLX', 'BAC', 'ABBV', 'CRM', 'ORCL', 'CVX', 'KO', 'AMD', 'MRK',
        'ADBE', 'PEP', 'TMO', 'ACN', 'LIN', 'CSCO', 'MCD', 'ABT', 'WFC', 'PM',
        'GE', 'QCOM', 'TXN', 'IBM', 'INTU', 'CAT', 'VZ', 'DHR', 'CMCSA', 'AMGN',
        'ISRG', 'NOW', 'PFE', 'NEE', 'AMAT', 'DIS', 'SPGI', 'HON', 'UBER', 'RTX',
        'UNP', 'LOW', 'AXP', 'T', 'COP', 'BKNG', 'ELV', 'SYK', 'MS', 'PLD',
        'BLK', 'UPS', 'GILD', 'LRCX', 'BA', 'VRTX', 'PANW', 'ADI', 'DE', 'MDT',
        'TJX', 'LMT', 'GS', 'SCHW', 'CI', 'MMC', 'ADP', 'CB', 'MDLZ', 'C',
        'REGN', 'BMY', 'AMT', 'SO', 'BSX', 'PGR', 'SLB', 'BX', 'ETN', 'SBUX',
        'KLAC', 'MU', 'ZTS', 'FI', 'EQIX', 'DUK', 'EOG', 'SNPS', 'ICE', 'CME',
        'CDNS', 'WM', 'PH', 'NOC', 'INTC', 'GD', 'APD', 'MCK', 'ITW', 'PYPL',
        'MAR', 'CL', 'CMG', 'MSI', 'EMR', 'CSX', 'MCO', 'APH', 'AON', 'USB',
        'WELL', 'NXPI', 'TT', 'MMM', 'SHW', 'TDG', 'ADSK', 'COF', 'CVS', 'ECL',
        'HCA', 'NSC', 'FCX', 'ROP', 'FDX', 'GM', 'PCAR', 'AFL', 'AJG', 'ABNB',
        'MPC', 'PSX', 'TGT', 'CARR', 'ORLY', 'AZO', 'JCI', 'AIG', 'AMP', 'GIS',
        'MET', 'SRE', 'D', 'NEM', 'TEL', 'TRV', 'O', 'KMB', 'HLT', 'SPG',
        'VLO', 'CCI', 'CPRT', 'OXY', 'FICO', 'PAYX', 'MSCI', 'FTNT', 'AEP', 'MCHP',
        'RSG', 'BK', 'ALL', 'PCG', 'SYY', 'PSA', 'KR', 'CTAS', 'CTVA', 'BKR',
        'DHI', 'JBL', 'PRU', 'EA', 'DLR', 'TRGP', 'CMI', 'FAST', 'HES', 'HSY',
        'EW', 'KMI', 'RCL', 'CHTR', 'IT', 'KVUE', 'PEG', 'COR', 'OTIS', 'URI',
        'YUM', 'MNST', 'KDP', 'DD', 'DAL', 'GEHC', 'ODFL', 'EXC', 'LULU', 'GWW',
        'PWR', 'ACGL', 'IDXX', 'NDAQ', 'VMC', 'AME', 'GLW', 'A', 'XEL', 'CTSH',
        'FANG', 'ED', 'WAB', 'MLM', 'HWM', 'ROK', 'WMB', 'HIG', 'DXCM', 'LEN',
        'HAL', 'IQV', 'EXR', 'DOW', 'IR', 'STZ', 'ANSS', 'ROST', 'RMD', 'CCEP',
        'EIX', 'CBRE', 'BRO', 'ON', 'MTD', 'MPWR', 'WEC', 'PPG', 'ETR', 'GPN',
        'VRSK', 'VICI', 'CDW', 'AWK', 'EBAY', 'KEYS', 'WTW', 'LYB', 'STT', 'GDDY',
        'BLDR', 'HUM', 'TTWO', 'XYL', 'HPQ', 'FTV', 'AEE', 'AVB', 'DECK', 'AXON',
        'LH', 'EQR', 'ZBH', 'HUBB', 'EFX', 'DTE', 'IFF', 'TYL', 'BIIB', 'PHM',
        'VLTO', 'TSN', 'PPL', 'CHD', 'NUE', 'MTB', 'DFS', 'CAH', 'TSCO', 'WY',
        'K', 'WBD', 'CNP', 'ES', 'FITB', 'APTV', 'NTAP', 'FE', 'STE', 'HOLX',
        'BR', 'DG', 'TROW', 'STX', 'WST', 'RF', 'ZBRA', 'BALL', 'CCL', 'MOH',
        'HBAN', 'LUV', 'MKC', 'CFG', 'VTR', 'IRM', 'BAX', 'STLD', 'WDC', 'TDY',
        'ATO', 'FDS', 'GPC', 'EXPD', 'ESS', 'DOV', 'EL', 'DRI', 'UAL', 'MAA',
        'EXPE', 'CBOE', 'CMS', 'WAT', 'INVH', 'SYF', 'ALGN', 'IP', 'TER', 'NVR',
        'PTC', 'SWK', 'NTRS', 'LDOS', 'J', 'KEY', 'CLX', 'CINF', 'HPE', 'BBY',
        'PODD', 'TXT', 'CAG', 'EPAM', 'PKG', 'LVS', 'CPT', 'IEX', 'OMC', 'LYV',
        'JBHT', 'RVTY', 'CF', 'MOS', 'AKAM', 'HST', 'INCY', 'LNT', 'AMCR', 'POOL',
        'EVRG', 'PKI', 'ULTA', 'SWKS', 'BXP', 'ALB', 'PAYC', 'COO', 'MTCH', 'TPR',
        'TECH', 'JKHY', 'CTLT', 'NDSN', 'EMN', 'KIM', 'UDR', 'AES', 'CRL', 'NCLH',
        'CPB', 'CE', 'HII', 'BG', 'REG', 'TFX', 'IPG', 'BEN', 'FRT', 'GL',
        'APA', 'AIZ', 'FFIV', 'TAP', 'HSIC', 'WYNN', 'CHRW', 'RJF', 'VTRS', 'PNR',
        'MGM', 'NI', 'MRO', 'NRG', 'BBWI', 'ALLE', 'BWA', 'SOLV', 'HAS', 'WHR',
        'GNRC', 'PNW', 'HRL', 'FMC', 'IVZ', 'SJM', 'AOS', 'AAL', 'MKTX', 'KMX',
        'ENPH', 'DVN', 'LKQ', 'UHS', 'ROL', 'L', 'JNPR', 'QRVO', 'CZR', 'PARA',
        'NWSA', 'NWS', 'FOX', 'FOXA'
    ]
    print(f"✅ Retrieved {len(tickers)} S&P 500 tickers")
    return tickers

def get_nasdaq_tickers():
    """NASDAQ-100 ticker list"""
    tickers = [
        'AAPL', 'MSFT', 'NVDA', 'AMZN', 'META', 'GOOGL', 'GOOG', 'AVGO', 'TSLA', 'COST',
        'NFLX', 'AMD', 'ADBE', 'QCOM', 'CSCO', 'INTU', 'TXN', 'AMGN', 'CMCSA', 'ISRG',
        'AMAT', 'HON', 'BKNG', 'UBER', 'PANW', 'LRCX', 'ADP', 'VRTX', 'GILD', 'ADI',
        'SBUX', 'MU', 'KLAC', 'REGN', 'SNPS', 'INTC', 'CDNS', 'PYPL', 'NXPI', 'MELI',
        'MDLZ', 'CME', 'ASML', 'CRWD', 'MAR', 'CTAS', 'ADSK', 'ABNB', 'CSX', 'ORLY',
        'PCAR', 'FTNT', 'CHTR', 'ROP', 'MNST', 'DASH', 'TTD', 'WDAY', 'CPRT', 'ODFL',
        'AEP', 'ROST', 'MRVL', 'FAST', 'PAYX', 'EA', 'DXCM', 'KDP', 'CTSH', 'IDXX',
        'KHC', 'VRSK', 'EXC', 'LULU', 'GEHC', 'CSGP', 'CCEP', 'BKR', 'ZS', 'DDOG',
        'AZN', 'TEAM', 'XEL', 'FANG', 'ANSS', 'ON', 'BIIB', 'MCHP', 'CDW', 'TTWO',
        'GFS', 'MDB', 'ILMN', 'WBD', 'ARM', 'WBA', 'MRNA', 'DLTR', 'SMCI'
    ]
    print(f"✅ Retrieved {len(tickers)} NASDAQ-100 tickers")
    return tickers

# Load tickers
sp500_tickers = get_sp500_tickers()
nasdaq_tickers = get_nasdaq_tickers()

## Data Download Functions

In [ ]:
# ============================================================================
# DATA DOWNLOAD FUNCTIONS
# ============================================================================

def get_last_date_from_df(df):
    """Get the last date from a DataFrame"""
    if df is None or df.empty:
        return None
    return pd.Timestamp(df.index.max().date())

def download_incremental_data(tickers, start_date, end_date):
    """Download new data since start_date"""
    
    all_data = {}
    failed = []
    
    for ticker in tqdm(tickers, desc="Downloading"):
        try:
            data = yf.download(
                ticker,
                start=start_date,
                end=end_date,
                progress=False,
                show_errors=False
            )
            
            if not data.empty:
                all_data[ticker] = data
            else:
                failed.append(ticker)
            
            time.sleep(0.2)  # Rate limiting
            
        except Exception as e:
            failed.append(ticker)
    
    print(f"\n✅ Downloaded: {len(all_data)}/{len(tickers)} tickers")
    if failed:
        print(f"⚠️  Failed: {len(failed)} tickers")
    
    return all_data

def create_price_dataframes(all_data):
    """Create separate DataFrames for each price type"""
    
    price_dfs = {}
    
    for price_type in ['Close', 'Adj Close', 'Open', 'High', 'Low', 'Volume']:
        df = pd.DataFrame({
            ticker: data[price_type]
            for ticker, data in all_data.items()
            if price_type in data.columns
        })
        
        # Map to our naming convention
        key_map = {
            'Close': 'close',
            'Adj Close': 'adj_close',
            'Open': 'open',
            'High': 'high',
            'Low': 'low',
            'Volume': 'volume'
        }
        
        price_dfs[key_map[price_type]] = df
    
    return price_dfs

def merge_dataframes(existing_df, new_df):
    """Merge existing and new data, removing duplicates"""
    
    if existing_df is None or existing_df.empty:
        return new_df
    
    if new_df is None or new_df.empty:
        return existing_df
    
    # Concatenate
    combined = pd.concat([existing_df, new_df])
    
    # Remove duplicate dates (keep last)
    combined = combined[~combined.index.duplicated(keep='last')]
    
    # Sort by date
    combined = combined.sort_index()
    
    return combined

print("✅ Data download functions defined")

## Main Update Function

In [ ]:
# ============================================================================
# MAIN UPDATE FUNCTION
# ============================================================================

def update_index_data(service, index_name, tickers, folder_name):
    """
    Complete update workflow for an index:
    1. Download existing CSVs from Google Drive
    2. Check last date
    3. Download incremental data
    4. Merge and upload back to Drive
    """
    
    print(f"\n{'='*70}")
    print(f"UPDATING {index_name}")
    print(f"{'='*70}\n")
    
    # Get or create folder structure
    base_folder_id = find_or_create_folder(service, DRIVE_FOLDER_NAME)
    index_folder_id = find_or_create_folder(service, folder_name, base_folder_id)
    
    # Check for existing adj_close file to determine last date
    adj_close_filename = f"{folder_name.lower()}_adj_close.csv"
    existing_file = find_file_in_folder(service, adj_close_filename, index_folder_id)
    
    last_date = None
    
    if existing_file:
        print(f"📁 Found existing data: {adj_close_filename}")
        existing_df = download_csv_from_drive(
            service,
            existing_file['id'],
            adj_close_filename
        )
        last_date = get_last_date_from_df(existing_df)
        print(f"📅 Last date in data: {last_date.strftime('%Y-%m-%d')}")
    else:
        print(f"🆕 No existing data found - will download full history")
        last_date = pd.Timestamp(START_DATE)
    
    # Check if update needed
    today = pd.Timestamp(datetime.today().date())
    
    if last_date >= today:
        print(f"✅ Data is current - no update needed!")
        return
    
    days_behind = (today - last_date).days
    print(f"\n📊 Need to update {days_behind} days of data")
    
    # Download incremental data
    start_date = (last_date + timedelta(days=1)).strftime('%Y-%m-%d')
    end_date = today.strftime('%Y-%m-%d')
    
    print(f"\n⏬ Downloading data from {start_date} to {end_date}...")
    new_data = download_incremental_data(tickers, start_date, end_date)
    
    if not new_data:
        print("⚠️  No new data available")
        return
    
    # Create price DataFrames
    print(f"\n📊 Processing new data...")
    new_price_dfs = create_price_dataframes(new_data)
    
    # Update each file type
    print(f"\n⏫ Updating files in Google Drive...\n")
    
    for price_type in PRICE_TYPES:
        filename = f"{folder_name.lower()}_{price_type}.csv"
        
        # Find existing file
        existing_file = find_file_in_folder(service, filename, index_folder_id)
        
        # Download existing data if it exists
        existing_df = None
        if existing_file:
            existing_df = download_csv_from_drive(
                service,
                existing_file['id'],
                filename
            )
        
        # Merge with new data
        if price_type in new_price_dfs:
            updated_df = merge_dataframes(existing_df, new_price_dfs[price_type])
            
            # Upload to Drive
            file_id = existing_file['id'] if existing_file else None
            upload_csv_to_drive(
                service,
                updated_df,
                filename,
                index_folder_id,
                file_id
            )
            
            print(f"   Shape: {updated_df.shape}, Date range: {updated_df.index.min().date()} to {updated_df.index.max().date()}")
    
    print(f"\n✅ {index_name} update complete!")

print("✅ Main update function defined")

## Run Updates

In [ ]:
# ============================================================================
# RUN UPDATES FOR BOTH INDICES
# ============================================================================

print("\n" + "="*70)
print("STARTING INCREMENTAL UPDATE")
print("="*70)

# Update S&P 500
update_index_data(
    drive_service,
    "S&P 500",
    sp500_tickers,
    "SP500"
)

# Update NASDAQ-100
update_index_data(
    drive_service,
    "NASDAQ-100",
    nasdaq_tickers,
    "NASDAQ"
)

print("\n" + "="*70)
print("✅ ALL UPDATES COMPLETE!")
print("="*70)
print(f"\n📁 Files updated in Google Drive: {DRIVE_FOLDER_NAME}/")
print("\n🎯 Next run will only download data since last update!")
print("⏱️  Estimated incremental runtime: 2-5 minutes")